# ・QCoin on IBMQ (Qiskit)

### Import modules. You need to install qiskit. Please refer to https://qiskit.org/documentation/install.html

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [3]:
import qiskit as qk
from qiskit import IBMQ, BasicAer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_overview

### Required IBMQ account from here. Please copy and paste your token into "mytoken".

In [5]:
IBMQ.update_account()

mytoken = ## your token ##
IBMQ.save_account(mytoken)

provider = IBMQ.load_account()
backends = provider.backends()
backends

Credentials already present. Set overwrite=True to overwrite.


Found 1 credentials.
No credentials available for updating could be found. No action will be performed.
Warnings:
The stored account with url "https://auth.quantum-computing.ibm.com/api" is already an IBM Q Experience v2 account.


/Users/NaoharuShimada/Library/Python/3.6/lib/python/site-packages/qiskit/providers/models/backendconfiguration.py:337: UserWarning: `dt` and `dtm` now have units of seconds(s) rather than nanoseconds(ns).
  warnings.warn('`dt` and `dtm` now have units of seconds(s) rather '


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [6]:
def Func_1st(ave, experiments, backend, shots, G_k) :
    output = "Query : (1+2×" + str(G_k) + ")×" + str(shots) + " ="
    print(output, (2*G_k+1)*shots)
    
    # Build quantum circuits
    circuits = []
    for i in range(experiments):
        theta = math.asin(math.sqrt(ave))*2
        #print(theta)

        # Define the Quantum and Classical Registers
        q = QuantumRegister(1)
        c = ClassicalRegister(1)

        # Build the circuits
        middle = QuantumCircuit(q, c)
        
        # 1st 0ne
        middle.u3(theta,0,0,q[0])
        middle.barrier()
        
        # 2nd~ AA
        for j in range(G_k):
            middle.z(q[0])
            middle.u3(-theta,0,0,q[0])
            middle.x(q[0])
            middle.z(q[0])
            middle.x(q[0])
            middle.u3(theta,0,0,q[0])
            middle.barrier()

        # Measure the 1st qubit
        meas = QuantumCircuit(q, c)
        meas.measure(q[0],c[0])

        circuits.append(middle + meas)
    
    print(circuits[1].draw())
    
    
    # Run on Quantum computer
    F_list = []
    error = 0.0
    print("")
    print("The backend is " + backend.name())
    job = execute(circuits, backend = backend, shots=shots)
    job_monitor(job)
    result = job.result()

    for i in range(experiments):
        counts = result.get_counts(circuits[i])
        if (len(counts)==1):
            for key in counts:
                if (key=='0'):
                    f = 0.0
                else:
                    f = 1.0
        else :   
            f = (shots-counts['0'])/shots
        F_list.append(f)
        print("Target:", ave, ",  Estimate: ", f)
        error += abs(f-ave)

    print("-------------------------------------------------------------------------------------------")
    print("Average abs error on quantum computer: ", error/experiments)
    return F_list

In [7]:
def Func_2nd(ave, experiments, backend, shots, G_k, F_list) :
    k = int(math.log2(G_k)) + 2
    print("k =", k)
    output = "Query : (1+2×(2×" + str(G_k) + "-1))×" + str(shots) + " ="
    print(output, (2*(2*G_k-1)+1)*shots)

    F = F_list
    experiments = len(F)

    delta = math.sin(math.pi/2.0/(2.0*G_k))/2.0 # error is ± sin(π/2/(2^k)) / 2
    print("δ =", delta)

    # Build quantum circuits
    circuits = []
    lower_limit = []
    for i in range(experiments):
        lower_limit.append(max(0.0,F[i]-delta))
        theta = math.asin(ave-lower_limit[i])*2

        # Define the Quantum and Classical Registers
        q = QuantumRegister(1)
        c = ClassicalRegister(1)

        # Build the circuits
        middle = QuantumCircuit(q, c)

        # Grover
        for j in range(G_k):
            middle.z(q[0])
            middle.u3(-theta,0,0,q[0])
            #middle.x(q[0])
            middle.z(q[0])
            #middle.x(q[0])
            middle.u3(theta,0,0,q[0])
            middle.barrier()

        # Measure 1st qubit
        meas = QuantumCircuit(q, c)
        #meas.barrier()
        meas.measure(q[0],c[0])

        circuits.append(middle + meas)
    print(circuits[1].draw())     
    
    # IBMQ 4
    job = execute(circuits, backend = backend, shots=shots)
    job_monitor(job)
    result = job.result()
    
    F_list = []
    error = 0.0
    for i in range(experiments):
        counts = result.get_counts(circuits[i])
        if (len(counts)==1):
            for key in counts:
                if (key=='0'):
                    f = 0.0
                else:
                    f = 1.0
        else :   
            f = (shots-counts['0'])/shots
        f = lower_limit[i] + math.sin(math.asin(math.sqrt(f))/(2*G_k))
        if (f>F[i]+delta):
            f = F[i]+delta
        F_list.append(f)
        print("Target:", ave, ",  Estimate: ", f)
        error += abs(f-ave)
    
    print("-------------------------------------------------------------------------------------------")
    print("Average abs error on quantum computer: ", error/experiments)
    return F_list

### Input variables. You can specify target mean value "ave" (0.0~1.0). The circuit is run by "experiments" times.

In [8]:
# input variables
ave = 0.50             # target_value
experiments = 75       # How many trials 
backend = backends[0]  # 0 for Simulator, 1 for ibmqx2
shots = 4              # the number of trials in each step (="L" in Algorithm 1 of our paper)

### You may execute fuctions defined above sequentially, for example: 
### Func_1st(~, 4, 0) → Func_2nd(~, 4, 1) → Func_2nd(~, 4, 2) → Func_2nd(~, 4, 4) → ...
### Func_1st is compatible to 1st step in Altorithm 1 (refer to our paper). Func_2nd is to the other steps.
### Quantum circuits drawn below are the same as Figure 9. Resulting error may be reduced forwarding the steps.

In [9]:
F_list = Func_1st(ave, experiments, backend, shots, 0)

Query : (1+2×0)×4 = 4
         ┌──────────────┐ ░ ┌─┐
q1_0: |0>┤ U3(pi/2,0,0) ├─░─┤M├
         └──────────────┘ ░ └╥┘
 c1_0: 0 ════════════════════╩═
                               

The backend is ibmq_qasm_simulator
Job Status: job has successfully run
Target: 0.5 ,  Estimate:  0.75
Target: 0.5 ,  Estimate:  0.0
Target: 0.5 ,  Estimate:  0.5
Target: 0.5 ,  Estimate:  0.25
Target: 0.5 ,  Estimate:  1.0
Target: 0.5 ,  Estimate:  0.5
Target: 0.5 ,  Estimate:  1.0
Target: 0.5 ,  Estimate:  0.25
Target: 0.5 ,  Estimate:  0.25
Target: 0.5 ,  Estimate:  0.25
Target: 0.5 ,  Estimate:  0.75
Target: 0.5 ,  Estimate:  0.5
Target: 0.5 ,  Estimate:  0.5
Target: 0.5 ,  Estimate:  0.75
Target: 0.5 ,  Estimate:  0.5
Target: 0.5 ,  Estimate:  0.25
Target: 0.5 ,  Estimate:  0.75
Target: 0.5 ,  Estimate:  1.0
Target: 0.5 ,  Estimate:  0.75
Target: 0.5 ,  Estimate:  0.75
Target: 0.5 ,  Estimate:  0.25
Target: 0.5 ,  Estimate:  0.25
Target: 0.5 ,  Estimate:  0.5
Target: 0.5 ,  Estimate:  0.5
Target: 0.5 

In [10]:
F_list = Func_2nd(ave, experiments, backend, shots, 1, F_list)

k = 2
Query : (1+2×(2×1-1))×4 = 12
δ = 0.35355339059327373
          ┌───┐┌───────────────┐┌───┐┌──────────────┐ ░ ┌─┐
q76_0: |0>┤ Z ├┤ U3(-pi/3,0,0) ├┤ Z ├┤ U3(pi/3,0,0) ├─░─┤M├
          └───┘└───────────────┘└───┘└──────────────┘ ░ └╥┘
 c76_0: 0 ═══════════════════════════════════════════════╩═
                                                           
Job Status: job has successfully run
Target: 0.5 ,  Estimate:  0.39644660940672627
Target: 0.5 ,  Estimate:  0.35355339059327373
Target: 0.5 ,  Estimate:  0.14644660940672627
Target: 0.5 ,  Estimate:  0.49999999999999994
Target: 0.5 ,  Estimate:  0.905265654509247
Target: 0.5 ,  Estimate:  0.5291300417718161
Target: 0.5 ,  Estimate:  0.905265654509247
Target: 0.5 ,  Estimate:  0.6035533905932737
Target: 0.5 ,  Estimate:  0.49999999999999994
Target: 0.5 ,  Estimate:  0.6035533905932737
Target: 0.5 ,  Estimate:  0.39644660940672627
Target: 0.5 ,  Estimate:  0.14644660940672627
Target: 0.5 ,  Estimate:  0.5291300417718161
Target: 0.5 , 

In [11]:
F_list = Func_2nd(ave, experiments, backend, shots, 2, F_list)

k = 3
Query : (1+2×(2×2-1))×4 = 28
δ = 0.1913417161825449
           ┌───┐┌──────────────────┐┌───┐┌─────────────────┐ ░ ┌───┐»
q151_0: |0>┤ Z ├┤ U3(-0.68913,0,0) ├┤ Z ├┤ U3(0.68913,0,0) ├─░─┤ Z ├»
           └───┘└──────────────────┘└───┘└─────────────────┘ ░ └───┘»
 c151_0: 0 ═════════════════════════════════════════════════════════»
                                                                    »
«        ┌──────────────────┐┌───┐┌─────────────────┐ ░ ┌─┐
«q151_0: ┤ U3(-0.68913,0,0) ├┤ Z ├┤ U3(0.68913,0,0) ├─░─┤M├
«        └──────────────────┘└───┘└─────────────────┘ ░ └╥┘
«c151_0: ════════════════════════════════════════════════╩═
«                                                          
Job Status: job has successfully run
Target: 0.5 ,  Estimate:  0.4639239383267021
Target: 0.5 ,  Estimate:  0.5448951067758186
Target: 0.5 ,  Estimate:  0.3377883255892712
Target: 0.5 ,  Estimate:  0.5674773289199758
Target: 0.5 ,  Estimate:  0.8444501305467537
Target: 0.5 ,  Estimate:  0.33

In [12]:
F_list = Func_2nd(ave, experiments, backend, shots, 4, F_list)

k = 4
Query : (1+2×(2×4-1))×4 = 60
δ = 0.09754516100806412
           ┌───┐┌──────────────────┐┌───┐┌─────────────────┐ ░ ┌───┐»
q226_0: |0>┤ Z ├┤ U3(-0.10535,0,0) ├┤ Z ├┤ U3(0.10535,0,0) ├─░─┤ Z ├»
           └───┘└──────────────────┘└───┘└─────────────────┘ ░ └───┘»
 c226_0: 0 ═════════════════════════════════════════════════════════»
                                                                    »
«        ┌──────────────────┐┌───┐┌─────────────────┐ ░ ┌───┐»
«q226_0: ┤ U3(-0.10535,0,0) ├┤ Z ├┤ U3(0.10535,0,0) ├─░─┤ Z ├»
«        └──────────────────┘└───┘└─────────────────┘ ░ └───┘»
«c226_0: ════════════════════════════════════════════════════»
«                                                            »
«        ┌──────────────────┐┌───┐┌─────────────────┐ ░ ┌───┐»
«q226_0: ┤ U3(-0.10535,0,0) ├┤ Z ├┤ U3(0.10535,0,0) ├─░─┤ Z ├»
«        └──────────────────┘└───┘└─────────────────┘ ░ └───┘»
«c226_0: ════════════════════════════════════════════════════»
«                       